
---

### This 📘 notebook needs to be run from VS Code with [these prerequisites](https://github.com/johnmaeda/SK-How-Tos#-prerequisites-to-getting-started-with-the-notebooks-in-this-repo).

How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# 📦💨 Packaging Semantic Functions For Sharing

One of the ways that Semantic Kernel shines is the way you can easily reuse semantic functions. There's a simple folder structure we use that's described in greater detail [here](https://learn.microsoft.com/en-us/semantic-kernel/howto/semanticfunctions), but let's do a quick review of how Semantic Skills and Semantic Functions can be stored as conventional files.

## Semantic Functions

Multiple semantic functions can be shared as text files within enclosing folders to be packaged within a skill folder

```
📁 A-Skill
│
└─── 📁 A-Function
|    |
│    └─── skprompt.txt
│    └─── config.json
│   
└─── 📁 Another-Function
     |
     └─── skprompt.txt
     └─── config.json
```
The format of the `config.json` file is [here](https://learn.microsoft.com/en-us/semantic-kernel/howto/configuringfunctions) but if its not included, a default behavior will be used.

## Semantic Skills

Multiple semantic skills usually get placed inside a directory called `skills` and the way you access a semantic function is through the name of the skill, and then the function name.

```
📁 skills
│
└─── 📁 A-Skill
│   
└─── 📁 B-Skill
│   
└─── 📁 C-Skill
```

For this notebook, there are a few semantic skills available in the `skills` directory.

## Step 1️: Let's get started by instantiating a kernel

You've already set up your API key information, so this should be an easy ▶️ and you're good to go.

In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.8.48.1-preview

😱 **Get an error message?** The [first notebook](../s1e1-ez-starter-notebook/notebook.ipynb) walks you through this process so you should be all set. But if you're still stuck, go to https://aka.ms/sk/discord where we have realtime support.

## Step 2: Access a semantic function stored in a folder from the Semantic Kernel

We will be using the `Joke` semantic function that is accessible within the `FunSkill` subdirectory of `skills`

```
📁 skills
│
└─── 📁 FunSkill
|    |
│    └─── 📁 Joke
│    └─── 📁 Limerick
│    └─── 📁 Excuses
└─── 📁 ChatSkill
```

### Step 2.1: Tell me a joke

And with that, let's get the LLM AI to tell us a quick 😂 joke.

In [ ]:
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the FunSkill from the Skills Directory
var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

// The default input variable 
var input = "time travel to dinosaur age";

// Run the Function called Joke with the default parameter of $input
var result = await kernel.RunAsync(input, skill["Joke"]);

// Return the result to the Notebook
Console.WriteLine(result);

### Step 2.2: Providing multiple inputs to a semantic function

There are actually two parameters that are used by `FunSkill.Joke`. One is `$input` and the other is `$audience_type`. We can deliver this broader context to the semantic function by assigning the `ContextVariables`, and then run the semantic function again with the broader context that's provided than just a single `$input`:

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

// Reload the FunSkill from the Skills Directory in case you are changing it for fun

var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

var myContext = new ContextVariables(); 
myContext.Set("input", "going to the department store"); 
myContext.Set("audience_type", "snobby people"); 

var myResult = await kernel.RunAsync(myContext,skill["Joke"]);

Console.WriteLine(myResult);


### Step 2.3: Every function doesn't need to use `$input`

Using the default `$input` named context variable is optional. For example, `FunSkill.Limerick` has two context variables `$name` and `$whoisname` to be used as follows:

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

// Load the Limerick function from the FunSkill

var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

var myContext = new ContextVariables(); 
myContext.Set("name", "Ichiro Suzuki"); 
myContext.Set("who_is_name", "the great baseball player"); 

var myResult = await kernel.RunAsync(myContext,skill["Limerick"]);

Console.WriteLine(myResult);

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

Or stay a longer while and change the `config.json` files to see how the different semantic functions behave. Or completely rewrite one of the functions that's been made available.